In [84]:
import numpy as np

A = np.ones((3,3))
B = np.triu((3,3, 3))

C = A + B

dC = np.ones_like(C)
dA = np.sum(dC, axis=0)
dB = dC

# dA.shape == A.shape, dB.shape == B.shape,
A.shape == C.shape
for i, j in zip(A.shape, C.shape):
    print("not equal", i == j)

not equal True
not equal True


In [77]:
# Create a 1D array
arr = np.linspace(0, 100, 101)
arr = arr[:-1]
reshaped_arr = arr.reshape((2, 5, -1))
arr = np.ones(10)
# Reshape the array to a 2D array with 2 rows and 3 columns

reshaped_arr + arr, reshaped_arr

(array([[[  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.],
         [ 11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.],
         [ 21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.],
         [ 31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.],
         [ 41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.]],
 
        [[ 51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.],
         [ 61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.],
         [ 71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.],
         [ 81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.],
         [ 91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99., 100.]]]),
 array([[[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
         [10., 11., 12., 13., 14., 15., 16., 17., 18., 19.],
         [20., 21., 22., 23., 24., 25., 26., 27., 28., 29.],
         [30., 31., 32., 33., 34., 35., 36., 37., 38., 39.],
         [40., 41., 42., 43., 44., 45., 

In [192]:
import torch

A = torch.ones((5, 4, 10), requires_grad=True)

B = torch.ones((10,5, 5))
B = torch.triu(B)
B.requires_grad=True

A.retain_grad()
B.retain_grad()

C = A @ B
grad_output = torch.ones_like(C)
C.backward(grad_output, retain_graph=True)

C.shape
dA = grad_output @ B.T
dA == A.grad
C.shape, A.shape, B.shape 

RuntimeError: The size of tensor a (5) must match the size of tensor b (10) at non-singleton dimension 0